In [4]:
import os
import itertools

In [11]:
root = os.path.join(os. getcwd(), 'data')
repos = [x for x in os.listdir(root)]
repo_pairs = itertools.product(repos, repeat=2)
repo_pairs_no_diagonal = [(x, y) for (x, y) in tuple(repo_pairs) if x != y]

In [ ]:
# remove package names, forked from CCFinder RJ1
# e.g.: "abi.encode(payload)" => "encode(payload)"
def transformation_rule_1(line):
    pass

# convert to compound block, forked from CCFinder RC6/RJ6
# e.g.: "if(True) return;" => "if(True) {return;}"
def transformation_rule_3(line):
    pass

# replace all parameters from [VARIABLE_NAME] to $p
def parameter_replacement(line):
    pass


In [52]:
import re

def remove_comment(text):
    string = re.sub(re.compile("/\*.*?\*/",re.DOTALL ), "", text) 
    # remove all occurrences streamed comments (/*COMMENT */) from string
    string = re.sub(re.compile("//.*?\n" ), "", string) 
    # remove all occurrence single-line comments (//COMMENT\n ) from string
    return string

dictionary_space = (" ", "\t", "\n")
def remove_space(text):
    lines = []
    for line in text.split("\n"):
        lines.append(" ".join([word.strip() for word in line.split(" ") if word.strip()]))
    #text = "\n".join(lines)
    text = "\n".join([line.strip("".join(dictionary_space)) for line in lines])
    return text


# remove accessibility keywords, forked from CCFinder RC5
# e.g.: "function owner() public view returns (address) {"
# => "function owner() returns (address) {"
dictionary_function_keywords = (
    "public", "private", "external", "internal", "view", "payable", "pure", 
    "indexed", # for event
    "override", "virtual" # for polymorphism
)
def transformation_rule_2(text):
    pattern = re.compile("|".join(
        [keyword for keyword in dictionary_function_keywords]), re.DOTALL)
    return re.sub(pattern, "", text) 

dictionary_operator = (
    "\(", "\)", "\[", "\]", "\{", "\}",
    "==", "!=", ">=", "<=", "\>", "\<", "\!", "\=", 
    "=>", "\.", "\|\|", "\&\&"
    "\+", "\-", "\*", "\%", "\\\\", # div
    ";", "\+\+", "\-\-"
)
def insert_white_space(text):
    indices = [m.start() for m in re.finditer("|".join(
        [operator for operator in dictionary_operator]), text)]
    chunks = []
    target = text
    for i in reversed(range(len(indices))):
        chunks.append(" " + target[indices[i]] + " " + target[indices[i]+1:] )
        chunks.append(target[:indices[i]])
        target = chunks.pop()
    chunks.reverse()
    return "".join(chunks)

    

In [53]:
file_name = "TokenManager.sol"
output_file_name = "TokenManager.sol.out"
with open(file_name, encoding="utf-8") as f:
    content = f.read()

content = remove_comment(content)

content = transformation_rule_2(content)
content = insert_white_space(content)
content = remove_space(content)

with open(output_file_name, "w") as f:
    f.write(content)